In [1]:
import pandas as pd
import numpy as np
from imdbpie import Imdb

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [ ]:
df = pd.DataFrame

In [35]:
#movie_count = 0 
movie_count = 216
# for i in range(10000):
for i in range(2580, 10000):
    id_ = 'tt'+str(0)+str(np.random.randint(0,4))+''.join(["%s" % np.random.randint(0, 9) for num in range(5)])
    print id_
    if imdb.title_exists(id_):
        try:
            title = imdb.get_title_by_id(id_)
            if title.type=='feature':
                if title.plot_outline!=None:
                    df.loc[i,'rating']=title.rating
                    df.loc[i,'title']=title.title
                    df.loc[i,'year']=title.year
                    for z in range(len(title.directors_summary)):
                        df.loc[i, 'director'+str(z)]=str(title.directors_summary[z]).rsplit("'")[1]
                    df.loc[i, 'runtime']= title.runtime
                    for y in range(len(title.genres)):
                        df.loc[i, 'genres'+str(y)]=title.genres[y]
                    for x in range(len(title.cast_summary)):
                        df.loc[i, 'actor' +str(x)]= str(title.cast_summary[x]).rsplit("'")[1]
                    df.loc[i, 'tagline']= title.tagline
                    df.loc[i, 'outline']= title.plot_outline
                    movie_count+=1
                    print movie_count
                    if movie_count == 500:
                        break
                else:
                    pass
            else:
                pass
        except:
            pass
    
print df

tt0022121
tt0013357
tt0287624
215
tt0027106
216
tt0174624
tt0022250
tt0077684
tt0287374
tt0161762
tt0268671


KeyboardInterrupt: 

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 1565 to 2579
Data columns (total 18 columns):
rating       190 non-null float64
title        216 non-null object
year         216 non-null float64
director0    214 non-null object
runtime      193 non-null float64
genres0      210 non-null object
actor0       210 non-null object
actor1       210 non-null object
actor2       210 non-null object
actor3       208 non-null object
tagline      86 non-null object
outline      210 non-null object
genres1      120 non-null object
genres2      44 non-null object
genres3      23 non-null object
director1    9 non-null object
genres4      3 non-null object
genres5      1 non-null object
dtypes: float64(3), object(15)
memory usage: 32.1+ KB


In [39]:
df.to_csv('initial_scrape.csv', sep=';', encoding ='utf-8')

In [25]:
(df['title'].value_counts()).head()

Madrid en el año 2000                      2
Mera Yaar Mera Dushman                     1
It'll Be Alright on the Christmas Night    1
Love in Paradise                           1
Vent d'est                                 1
Name: title, dtype: int64

In [36]:
df.drop_duplicates

KeyboardInterrupt: 

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
p_20 = np.percentile(df['rating'], 20)
print p_20
p_40 = np.percentile(df['rating'], 40)
print p_40
p_60 = np.percentile(df['rating'], 60)
print p_60
p_80 = np.percentile(df['rating'], 80)
print p_80

In [ ]:
for i in range(len(df['rating'])):
    if df.loc[i, 'rating']<p_20:
        df.loc[i, 'cat'] =1
    elif df.loc[i, 'rating']<p_40:
        df.loc[i, 'cat'] =2
    elif df.loc[i, 'rating']<p_60:
        df.loc[i, 'cat'] =3
    elif df.loc[i, 'rating']<p_80:
        df.loc[i, 'cat'] =4
    else:
        df.loc[i, 'cat'] =5

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.hist(df['rating'], bins=13)
plt.title('Distribution of sample movie ratings')
plt.show()

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
plt.hist(df['year'], bins =25)
plt.title("Distribution of sample by year")
plt.show()

In [ ]:
plt.hist(df['runtime'], bins =25)
plt.title("Distribution of sample runtimes in seconds")
plt.show()

In [ ]:
df['tagline'] = df['tagline'].fillna(value='I')

In [ ]:
cv = CountVectorizer(max_features=50, stop_words='english')
data = cv.fit_transform(df.tagline).todense()
columns = [c for c in cv.get_feature_names()]
taglinedf = pd.DataFrame(data, columns=columns)
taglinedf.columns = ['tag_' + str(col) for col in taglinedf.columns]
taglinedf.head()

In [ ]:
data = cv.fit_transform(df.outline).todense()
columns = [c for c in cv.get_feature_names()]
outlinedf = pd.DataFrame(data, columns=columns)
outlinedf.columns = ['out_' + str(col) for col in outlinedf.columns]
outlinedf.head()

In [ ]:
genre1 = pd.get_dummies(df.loc[:,'genres0'], prefix = 'gen_')
genre2 = pd.get_dummies(df.loc[:,'genres1'], prefix = 'gen_')
genre3 = pd.get_dummies(df.loc[:,'genres2'], prefix = 'gen_')
genre4 = pd.get_dummies(df.loc[:,'genres3'], prefix = 'gen_')
genre12 = genre1.add(genre2, fill_value=0)
genre123 = genre12.add(genre3, fill_value = 0)
genre = genre123.add(genre4, fill_value = 0)

In [ ]:
director1 = pd.get_dummies(df.loc[:,'director0'], prefix = 'dir_')
director2 = pd.get_dummies(df.loc[:,'director1'], prefix = 'dir_')
director = director1.add(director2, fill_value = 0)

In [ ]:
actor1 = pd.get_dummies(df.loc[:,'actor0'], prefix = 'act_')
actor2 = pd.get_dummies(df.loc[:,'actor1'], prefix = 'act_')
actor3 = pd.get_dummies(df.loc[:,'actor2'], prefix = 'act_')
actor4 = pd.get_dummies(df.loc[:,'actor3'], prefix = 'act_')
actor12 = actor1.add(actor2, fill_value=0)
actor123 = actor12.add(actor3, fill_value = 0)
actor = actor123.add(actor4, fill_value = 0)

In [ ]:
df_new = df.join(taglinedf)
df_new = df_new.join(outlinedf)
df_new = df_new.join(genre)
df_new = df_new.join(director)
df_new = df_new.join(actor)

In [ ]:
y=df['cat']

In [ ]:
X = df.loc[:, 'tag_':]

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
treereg = DecisionTreeClassifier()
treereg.fit(X_train, y_train)

In [ ]:
preds = treereg.predict(X_test)

In [ ]:
plt.scatter(y_test, preds)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
